
---

### Reminder: This 📘 `.NET Interactive` notebook needs to be run from VS Code with [these prerequisites](../PREREQS.md).

#### How to use this notebook: 

* Just read the text and scroll along until you run into code blocks.
* Code blocks have computer code inside them — hover over the block and you can run the code.
* Run the code by hitting the ▶️ "play" button to the left. If the code runs you'll see a ✔️. If not, you'll get a ❌.
* The output and status of the code block will appear just below itself — you need to scroll down further to see it.
* Sometimes a code block will ask you for input in a hard-to-notice dialog box 👆 at the top of your notebook window. 

---

# Bonus Recipe: 🕵️ Prompt Secrets
## 🧑‍🍳 Masterfully cook your prompts to perfection

There are plenty of resources available on the Internet to become an expert prompter. Well, that's not really true because LLM AIs are still relatively new and everyone's trying to figure it all out still. Here's three tips to better prompting that might get you started along your journey as a so-called "prompt engineer."

## Step 1: Instantiate a 🔥 kernel so we can start cooking

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.9.61.1-preview"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.KernelExtensions;
using System.IO;
using Microsoft.SemanticKernel.Configuration;
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.Orchestration;

IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder.Build();

// Grab the locally stored credentials from the settings.json file. Name the "backend" as "davinci" — assuming that you're using one of the davinci completion models. 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    kernel.Config.AddAzureOpenAITextCompletion("davinci", model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAITextCompletion("davinci", model, apiKey, orgId);

## Step 2: Go through the individual prompting tips in any order that you like

Note that we are using a more compact representation for setting up a semantic function and running it all in one code block. It's a different approach, but it has the same outcome as the other method.

### Tip #1: Always give more 🥑 context in the prompt

The context of your prompt is like the 🥑 fat in a meal. It's rich and indulgent — often times it's what makes the meal. LLMs work well when you give as much context as possible. That's usually done by simply giving an example for how something should get done. When you give just one example, that's called "one shot" training.

In [ ]:
var myInlineSemanticFunction = kernel.CreateSemanticFunction(@"
I am someone who puts their time and money where their mouth is.
For example: When I say I care about the environment, I donate to environmental causes.
For example: When I say {{$input}}
", maxTokens: 100, temperature: 0.4, topP: 1);
var result = await myInlineSemanticFunction.InvokeAsync("health matters to me");
Console.WriteLine(result);

The response is likely in line with the example, but it doesn't constrain to the idea of "donating to a cause." So instead, you can feed the prompt with more than one example — which is called "few shot" training.

In [ ]:
var myInlineSemanticFunction = kernel.CreateSemanticFunction(@"
I am someone who puts their time and money where their mouth is.
For example: When I say I care about the environment, I donate to environmental causes.
For example: When I say I know that traffic is a problem in my city, I donate to a traffic safety organization.
For example: When I say {{$input}}
", maxTokens: 100, temperature: 0.4, topP: 1);
var result = await myInlineSemanticFunction.InvokeAsync("health matters to me");
Console.WriteLine(result);

Now you're likely to see it say `I donate to ....` as the response. The more examples you provide, the more deterministic the output will veer towards. But keep in mind that sometimes if you give it too many examples, it can have the opposite effect.

Lastly, you can also just be lazy and give it no example at all. This is called "zero shot" training.

In [ ]:
var myInlineSemanticFunction = kernel.CreateSemanticFunction(@"
Because {{$input}} I need to:
    ", maxTokens: 100, temperature: 0.4, topP: 1);
var result = await myInlineSemanticFunction.InvokeAsync("health matters to me");
Console.WriteLine(result);

And the response can be pretty amazing. To see a wider range of responses, change the `temperature` parameter to `0.9` or change the `topP` to `0.1` — these parameters govern the "shape" of what the LLM AI will generate.

### Tip #2: Shorten an often-used prompt 

You can reduce the amount of tokens that get used by finding a similar but shorter version of a prompt That way if you’re using the prompt a lot, you’re not using up as many tokens in a cumulative sense. If you're wondering how many tokens will be used by your prompt, the [online tokenizer calculator](https://platform.openai.com/tokenizer) from OpenAI is eminently useful.

For example, the prompt:

```
Take the list of global cities that represent my favorite destinations:

START OF LIST
Los Angeles, Tokyo, Madagascar, Boston, Phoenix, Paris, Dusseldorf
END OF LIST

and alphabetize them carefully such that all proper rules of English are used
```

That prompt would use 60 tokens. 

In [ ]:
var myInlineSemanticFunction = kernel.CreateSemanticFunction(@"
Take the list of global cities that represent my favorite destinations:

START OF LIST
Los Angeles, Tokyo, Madagascar, Boston, Phoenix, Paris, Dusseldorf
END OF LIST

and alphabetize them carefully such that all proper rules of English are used", maxTokens: 100, temperature: 0.4, topP: 1);
var result = await myInlineSemanticFunction.InvokeAsync();
Console.WriteLine("60 token prompt ==> " + result);

Compare that with the same prompt but made shorter and only using 29 tokens:

```
Alphabetize:

===
Los Angeles, Tokyo, Madagascar, Boston, Phoenix, Paris, Dusseldorf
===

```

In [ ]:
var myInlineSemanticFunction = kernel.CreateSemanticFunction(@"
Alphabetize:

===
Los Angeles, Tokyo, Madagascar, Boston, Phoenix, Paris, Dusseldorf
===
", maxTokens: 100, temperature: 0.4, topP: 1);
var result = await myInlineSemanticFunction.InvokeAsync();
Console.WriteLine("29 token prompt ==> " + result);

From a cost perspective, the shorter prompt would pay off in the long run. Token length is an important consideration not only from the point of view of economics, but also because of fundamental limitations in how many tokens you can use for the model you've selected. We'll go into detail on this point further in the S1E6 notebook that's coming right up.

### Tip #3: Go for a two-fer or three-fer

Sometimes with one prompt you can get multiple things done. It's like asking a combo-prompt where you've made a request to solve a problem that has three or multiple facets to it. For example, you could execute two prompts in sequence to work on a similar question like, generating a summary of a text and using 77 input tokens: 

```
Analyze the following text to produce a summary for a second grader: 

I went to the quantum realm to discover a new kind 
of machine to cure my oldest daughter from a
long-standing ailment. It wasn't easy. I had to 
travel through time and space to reach the realm 
and collect magical ingredients that could be used 
to create the medicine.
```

And then processing a second prompt to generate a list of keywords that will use 75 input tokens:

```
Analyze the following text to produce an alphabetized list of keywords:

I went to the quantum realm to discover a new kind 
of machine to cure my oldest daughter from a
long-standing ailment. It wasn't easy. I had to travel 
through time and space to reach the realm 
and collect magical ingredients that could be 
used to create the medicine.
```

That's a total of 152 tokens used on two different completion calls to the LLM AI. Instead, both requests can be combined into a single prompt that only takes 87 input tokens:

```
Analyze the following text to produce: 1. Summary for a second grader, 2. Alphabetized list of keywords. 

I went to the quantum realm to discover a new 
kind of machine to cure my oldest daughter from a
long-standing ailment. It wasn't easy. I had to 
travel through time and space to reach the realm 
and collect magical ingredients that could be 
used to create the medicine.
```

Let's try that out and see if it really works:

In [ ]:
var myInlineSemanticFunction = kernel.CreateSemanticFunction(@"
Analyze the following text to produce: 1. Summary for a second grader, 2. Alphabetized list of keywords, 3. One word summary.

I went to the quantum realm to discover a new kind of machine to cure my oldest daughter from a
long-standing ailment. It wasn't easy. I had to travel through time and space to reach the realm 
and collect magical ingredients that could be used to create the medicine.

1.
", maxTokens: 150, temperature: 0.4, topP: 1);
var result = await myInlineSemanticFunction.InvokeAsync();
Console.WriteLine("87 token prompt ==> " + result);

So it produced three results from just one prompt. You can imagine using this technique to generate more results as well.

### Tip #4: Use "chain of thought" prompting

Of all the "tips" out there, the current fan-favorite is what's called "chain of thought" prompting as referenced in [this scientific paper](https://arxiv.org/abs/2201.11903). How does it work? 
Getting a model to use  ur own human tendency to "think out loud" has been shown to produce more 
accurate outcomes. So when prompting, give an explicit "thinking out loud" explanation for how a problem is to be solved with your related reasoning. You'll get a better response because there's more 🥑 context for the model to work with, and ultimately it will provide a better response. Just like when you ask someone to do an important task, and before they run off and do it you ask them to walk you through the steps. Right? It works with people. And it works with LLM AIs too.

In [ ]:
var myInlineSemanticFunction = kernel.CreateSemanticFunction(@"
I want to eat a mushroom pizza. To make the pizza I need to:
", maxTokens: 100, temperature: 0.1, topP: 1);
var result = await myInlineSemanticFunction.InvokeAsync();
Console.WriteLine(result);

The response generated should read similar to:

1. _Preheat the oven to 375 degrees Fahrenheit._
2. _Roll out the pizza dough and place it on a greased baking sheet._
3. _Spread a thin layer of tomato sauce over the dough._
4. _Top with sliced mushrooms, shredded cheese, and any other desired toppings._
5. _Bake for 15-20 minutes, or until the cheese is melted and the crust is golden brown._
6. _Remove from the oven and let cool_

We can ask it for common mistakes for this set of instructions by feeding it back into the prompt:

In [ ]:
var myInlineSemanticFunction = kernel.CreateSemanticFunction(@"
I want to eat a mushroom pizza. To make the pizza I need to:
1. Preheat the oven to 375 degrees Fahrenheit.
2. Roll out the pizza dough and place it on a greased baking sheet.
3. Spread a thin layer of tomato sauce over the dough.
4. Top with sliced mushrooms, shredded cheese, and any other desired toppings.
5. Bake for 15-20 minutes, or until the cheese is melted and the crust is golden brown.
6. Remove from the oven and let cool
Common mistakes in these steps include:
", maxTokens: 100, temperature: 0.1, topP: 1);
var result = await myInlineSemanticFunction.InvokeAsync();
Console.WriteLine(result);

And the list ensues:

1. _Not preheating the oven before baking the pizza._
2. _Not greasing the baking sheet before placing the dough on it._
3. _Not spreading the tomato sauce evenly over the dough._
4. _Overloading the pizza with toppings._
5. _Not baking the pizza long enough._
6. _Not allowing the pizza to cool before serving._

Some of the common mistakes are obvious. But some of them aren't -- like step 4 or 5, for instance. We can feed this list back into the prompt and ask it for an improved series of steps to make a pizza:

In [ ]:
var myInlineSemanticFunction = kernel.CreateSemanticFunction(@"
I want to eat a mushroom pizza. To make the pizza I need to:
1. Preheat the oven to 375 degrees Fahrenheit.
2. Roll out the pizza dough and place it on a greased baking sheet.
3. Spread a thin layer of tomato sauce over the dough.
4. Top with sliced mushrooms, shredded cheese, and any other desired toppings.
5. Bake for 15-20 minutes, or until the cheese is melted and the crust is golden brown.
6. Remove from the oven and let cool
Common mistakes in these steps include:
1. Not preheating the oven before baking the pizza.
2. Not greasing the baking sheet before placing the dough on it.
3. Not spreading the tomato sauce evenly over the dough.
4. Overloading the pizza with toppings.
5. Not baking the pizza long enough.
6. Not allowing the pizza to cool before serving.
Rethinking the instructions for making a pizza so that a perfect pizza can be made every time:
", maxTokens: 200, temperature: 0.3, topP: 1);
var result = await myInlineSemanticFunction.InvokeAsync();
Console.WriteLine(result);

The resulting pizza instructions read as follows:

1. _Preheat the oven to 375 degrees Fahrenheit._
2. _Grease a baking sheet and roll out the pizza dough onto it._
3. _Spread a thin layer of tomato sauce evenly over the dough._
4. _Top the pizza with a moderate amount of sliced mushrooms, shredded cheese, and any other desired toppings._
5. _Bake for 15-20 minutes, or until the cheese is melted and the crust is golden brown._
6. _Allow the pizza to cool for a few minutes before serving._

Which are superior to the original set of instructions that were provided. Just for comparison:

| Fast thinking | Slow thinking (Chain of thought) | Difference
|---|---|---|
| _Preheat the oven to 375 degrees Fahrenheit._ | _Preheat the oven to 375 degrees Fahrenheit._ | Same |
| _Roll out the pizza dough and place it on a greased baking sheet._ | _Grease a baking sheet and roll out the pizza dough onto it._ | Emphasizes greasing |
| _Spread a thin layer of tomato sauce over the dough._ | _Spread a thin layer of tomato sauce evenly over the dough._ | Emphasizes evenness |
| _Top with sliced mushrooms, shredded cheese, and any other desired toppings._ | _Top the pizza with a moderate amount of sliced mushrooms, shredded cheese, and any other desired toppings._ | Emphasizes moderation |
| _Bake for 15-20 minutes, or until the cheese is melted and the crust is golden brown._ | _Bake for 15-20 minutes, or until the cheese is melted and the crust is golden brown._ | Same |
| _Remove from the oven and let cool_ | _Allow the pizza to cool for a few minutes before serving._ | Recommends time |

# ⏭️ Next Steps

Run through more advanced examples in the notebooks that are available in our GitHub repo at [https://aka.ms/sk/repo](https://aka.ms/sk/repo).

[You're all done! Visit the main GitHub repo to check out what's new — because LLM AI is changing ever-so-rapidly!](https://aka.ms/sk/repo)

Or, stay a longer while and change the objective of a chain of thought prompt to see how your results compare when you get the LLM AI to "think out loud" a bit.